In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from zipfile import ZipFile
zfile = ZipFile('/content/drive/MyDrive/cellula data/Teeth DataSet.zip')
zfile.extractall('data')

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, ReLU
from keras.models import Sequential, Model
from keras import layers, models
import tensorflow as tf
import numpy as np
import os
import shutil

In [4]:
import shutil

shutil.move('/content/data/Teeth_Dataset/Testing/out', '/content')
shutil.move('/content/data/Teeth_Dataset/Testing/output', '/content')
shutil.move('/content/data/Teeth_Dataset/Testing/outputs', '/content')

'/content/outputs'

# Data Preprocessing

In [2]:
train_gen = ImageDataGenerator(rescale=1/255,
                               horizontal_flip=True,
                               zoom_range=0.3,
                               rotation_range=0.3)

test_gen = ImageDataGenerator(rescale=1/255)

In [3]:
train_data = train_gen.flow_from_directory('/content/data/Teeth_Dataset/Training',
                                            class_mode='categorical',
                                           target_size=(224, 224),
                                           batch_size=32)

test_data = test_gen.flow_from_directory('/content/data/Teeth_Dataset/Testing',
                                            class_mode='categorical',
                                           target_size=(224, 224),
                                           batch_size=32)
val_data = test_gen.flow_from_directory('/content/data/Teeth_Dataset/Validation',
                                            class_mode='categorical',
                                           target_size=(224, 224),
                                           batch_size=32)

Found 3087 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.


# Build the model

In [ ]:
def identity_block(x, filters, kernel_size=3):
    f1, f2, f3 = filters

    shortcut = x

    x = layers.Conv2D(f1, (1, 1), padding='valid')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(f2, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(f3, (1, 1), padding='valid')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Add()([shortcut, x])
    x = layers.Activation('relu')(x)

    return x

In [ ]:
def conv_block(x, filters, kernel_size=3, stride=2):
    f1, f2, f3 = filters

    shortcut = x

    x = layers.Conv2D(f1, (1, 1), strides=stride, padding='valid')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(f2, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(f3, (1, 1), padding='valid')(x)
    x = layers.BatchNormalization()(x)

    shortcut = layers.Conv2D(f3, (1, 1), strides=stride, padding='valid')(shortcut)
    shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([shortcut, x])
    x = layers.Activation('relu')(x)

    return x

In [ ]:
def ResNet50(input_shape=(224, 224, 3), num_classes=7):
    inputs = layers.Input(shape=input_shape)

    x = layers.ZeroPadding2D(padding=(3, 3))(inputs)

    # 1
    x = layers.Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    # 2
    x = conv_block(x, [64, 64, 256], kernel_size=3, stride=1)
    x = identity_block(x, [64, 64, 256], kernel_size=3)
    x = identity_block(x, [64, 64, 256], kernel_size=3)

    # 3
    x = conv_block(x, [128, 128, 512], kernel_size=3, stride=2)
    x = identity_block(x, [128, 128, 512], kernel_size=3)
    x = identity_block(x, [128, 128, 512], kernel_size=3)
    x = identity_block(x, [128, 128, 512], kernel_size=3)

    # 4
    x = conv_block(x, [256, 256, 1024], kernel_size=3, stride=2)
    x = identity_block(x, [256, 256, 1024], kernel_size=3)
    x = identity_block(x, [256, 256, 1024], kernel_size=3)
    x = identity_block(x, [256, 256, 1024], kernel_size=3)
    x = identity_block(x, [256, 256, 1024], kernel_size=3)
    x = identity_block(x, [256, 256, 1024], kernel_size=3)

    # 5
    x = conv_block(x, [512, 512, 2048], kernel_size=3, stride=2)
    x = identity_block(x, [512, 512, 2048], kernel_size=3)
    x = identity_block(x, [512, 512, 2048], kernel_size=3)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x, name='ResNet50')
    return model


model = ResNet50()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [17]:
hist = model.fit(train_data, epochs=50, validation_data=val_data, batch_size=32)

Epoch 1/50
97/97 ━━━━━━━━━━━━━━━━━━━━ 143s 827ms/step - accuracy: 0.2636 - loss: 2.7201 - val_accuracy: 0.1576 - val_loss: 2.5889
Epoch 2/50
97/97 ━━━━━━━━━━━━━━━━━━━━ 81s 507ms/step - accuracy: 0.3788 - loss: 1.6889 - val_accuracy: 0.1449 - val_loss: 3.9467
Epoch 3/50
97/97 ━━━━━━━━━━━━━━━━━━━━ 82s 502ms/step - accuracy: 0.4003 - loss: 1.6572 - val_accuracy: 0.1975 - val_loss: 2.6784
Epoch 4/50
97/97 ━━━━━━━━━━━━━━━━━━━━ 83s 511ms/step - accuracy: 0.4511 - loss: 1.5089 - val_accuracy: 0.2247 - val_loss: 2.4846
Epoch 5/50
97/97 ━━━━━━━━━━━━━━━━━━━━ 51s 500ms/step - accuracy: 0.4892 - loss: 1.3304 - val_accuracy: 0.2909 - val_loss: 2.2624
Epoch 6/50
97/97 ━━━━━━━━━━━━━━━━━━━━ 50s 483ms/step - accuracy: 0.5697 - loss: 1.1916 - val_accuracy: 0.2189 - val_loss: 4.9319
Epoch 7/50
97/97 ━━━━━━━━━━━━━━━━━━━━ 81s 484ms/step - accuracy: 0.6234 - loss: 1.0707 - val_accuracy: 0.2228 - val_loss: 3.0416
Epoch 8/50
97/97 ━━━━━━━━━━━━━━━━━━━━ 85s 511ms/step - accuracy: 0.6279 - loss: 1.0288 - val_acc

In [22]:
model.evaluate(test_data)

 1/33 ━━━━━━━━━━━━━━━━━━━━ 10s 325ms/step - accuracy: 0.7188 - loss: 0.7624

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - accuracy: 0.6885 - loss: 1.2122


[1.2126647233963013, 0.698443591594696]

In [19]:
model.save('teeth.keras')

In [21]:
shutil.move('/content/teeth.h5', '/content/drive/MyDrive/cellula data')
shutil.move('/content/teeth.keras', '/content/drive/MyDrive/cellula data')

'/content/drive/MyDrive/cellula data/teeth.keras'